In [1]:
#Libraries Used
import socket
import pickle
import time
import sys

In [2]:
#Client 1 has its public and private keys and public key of PKDA
with open('publicKey_client1.pkl','rb') as file:
    publicKey_client1 = pickle.load(file)
    
with open('privateKey_client1.pkl','rb') as file:
    privateKey_client1 = pickle.load(file)
    
with open('publicKey_PKDA.pkl','rb') as file:
    publicKey_PKDA = pickle.load(file)

In [3]:
#Evaluating the binary exponention 
#M^e (mod n)
#C^d (mod n)

def binary_modular_exponention(m,e,n):
    result = 1
    m = m % n
    if (m == 0):
        return 0
    while(e>0):
        if(e&1):
            result = (result*m) % n
        e = e>>1
        m = (m*m) % n
    return result

In [4]:
#Encryption Algorithm
def encrypt_data(encryption_key, message):
    cipher_text = ''
    for value in range(0, len(message)):
        encrypted_char = binary_modular_exponention(ord(message[value]), encryption_key[0], encryption_key[1])
        cipher_text += chr(encrypted_char)
    return cipher_text

In [5]:
#Decryption Algorithm
def decrypt_data(decryption_key, cipher_data):
    original_text = ''
    for value in range(0, len(cipher_data)):
        decrypted_char = binary_modular_exponention(ord(cipher_data[value]), decryption_key[0], decryption_key[1])
        original_text += chr(decrypted_char)
    return original_text

In [6]:
#Closing any previous socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.close()

In [7]:
#Defining the local host and different ports
host = 'local host'
server_port = 5000
client2_port = 5001
client1_port = 5002

#Defining the Socket Connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print('Connecting to Server....')
s.connect(('127.0.0.1', server_port))
print()
print('Connected to Server')

Connecting to Server....

Connected to Server


In [8]:
#Step 1 
# (1) Request || T1

#Connected to Server
msg = s.recv(1024)
print(msg.decode())

#Message Generation
t1 = time.time()
req_ts = 'client2'+'||'+str(t1)
s.send(req_ts.encode())
print('Request sent by client 1 :',req_ts)
print()
msg = s.recv(1024)

#Decode the message receieved from PKDA from Step 2
decrypted_text = decrypt_data(publicKey_PKDA, msg.decode())
print('Message received from server(after decryption) \n : ',decrypted_text)
print()

#Finding the public key of client to be connected
requested_public_key_e = int(decrypted_text.split('||')[0])
requested_public_key_n = int(decrypted_text.split('||')[1])
requested_public_key = [requested_public_key_e,requested_public_key_n]
print()
print('Public key of requested client :',requested_public_key)
print()
print('Sent Time Stamp T1: ', decrypted_text.split('||')[3])
print()
print('Recieved Time Stamp T1: ', str(time.time()))
print()

#Step 3
# E(PUb, [ IDA || N1])

#Connecting to Client 2
print()
print('Connecting to Client 2')
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('127.0.0.1', client2_port))
msg = s.recv(1024)
print(msg.decode())
print()

#Send the response as the data encrypted using the Public key of Client 2
IDA= 'client1'
N1 = str(time.time())
IDA_N1 = IDA + '||' + N1
print('Client 1 to Client 2(before encryption) : ', IDA_N1)
print()
#Message Encrypted which is to be send to Client 2
encrypted_text = encrypt_data(requested_public_key, IDA_N1)
print('Client 1 to Client 2(after encryption) : ', encrypted_text)
s.send(encrypted_text.encode())
print()
s.close()

#Seting up Socket Connection to listen to Client 2
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.bind(('', client1_port))
s.listen(5)
c, addr = s.accept()
print("CONNECTION FROM:", str(addr))
c.send(b"connected succcessfully to client1")
print()
msg = c.recv(1024) #Message from Step6
decrypted_text = decrypt_data(privateKey_client1, msg.decode())
print('Client 2 to Client 1(after decryption) : ',decrypted_text)

n2=decrypted_text.split('||')[1]
n1=decrypted_text.split('||')[0]

if(n1 == N1):
    print('Original n1 =',N1)
    print('Received n2 =', n1)
    print('Message Authentic')
else:
    print('Message found not Authentic')
    sys.exit(1)
print()  

#Step 7 Establishing connection with Client 2 finally
# (7) E(PUb, N2)
encrypted_text = encrypt_data(requested_public_key, n2)
c.send(encrypted_text.encode())
print('Connection Established!')

connected succcessfully to Public-Key Authority
Request sent by client 1 : client2||1635253162.6066768

Message received from server(after decryption) 
 :  5||221||client2||1635253162.6066768


Public key of requested client : [5, 221]

Sent Time Stamp T1:  1635253162.6066768

Recieved Time Stamp T1:  1635253162.6122663


Connecting to Client 2
Connected Succcessfully to Client2

Client 1 to Client 2(before encryption) :  client1||1635253162.6156664

Client 1 to Client 2(after encryption) :  ØKÑºy§§y¯3B!B3y¯!%¯yB¯¯¯4

CONNECTION FROM: ('127.0.0.1', 50654)

Client 2 to Client 1(after decryption) :  1635253162.6156664||1635253162.622589
Original n1 = 1635253162.6156664
Received n2 = 1635253162.6156664
Message Authentic

Connection Established!


In [9]:
#Printing the messages
count=1
while(count<=3):
    #Sending Messages
    message = 'Hi '+str(count)
    print('Client 1(this client) :', message)
    encrypted_text = encrypt_data(requested_public_key, message)
    print('Client 1 Encrypted Message(this Client): ', encrypted_text)
    print()
    c.send(encrypted_text.encode())
    
    count+=1
    
    #Receiving Messages
    msg = c.recv(1024)
    print('Client 2 says(Encrypted Message): ', msg.decode())
    decrypted_text = decrypt_data(privateKey_client1, msg.decode())
    print('Client 2 says : ', decrypted_text)
    print()
    
c.close()

Client 1(this client) : Hi 1
Client 1 Encrypted Message(this Client):  YÑy

Client 2 says(Encrypted Message):  N"]Dw]di
Client 2 says :  Got-it 1

Client 1(this client) : Hi 2
Client 1 Encrypted Message(this Client):  YÑ!

Client 2 says(Encrypted Message):  N"]Dw]d
Client 2 says :  Got-it 2

Client 1(this client) : Hi 3
Client 1 Encrypted Message(this Client):  YÑ3

Client 2 says(Encrypted Message):  N"]Dw]d
Client 2 says :  Got-it 3

